# Trabajando con fuentes de información

En esta práctica vamos a realizar un preanálisis de los datos de una fuente de información, manipularemos ficheros de texto que tomaremos como si hubieran sido obtenidos de una fuente de información desconocida, para posteriormente construir una fuente de Markov a partir de la misma. Obtendremos mensajes nuevos generados por la fuente y calcularemos la
entropía del texto que ha servido para modelar la fuente.

## Datos

Elegiremos algún [fichero en español del "Proyecto Gutenberg"](https://www.gutenberg.org/browse/languages/es) en formato `.txt`.

### Limpieza y preparación de los datos

Vamos a definir todas nuestras funciones y pasos como métodos de una clase llamada `FuenteDeInformacion`. El primer paso sería definir el constructor y un método que nos permita leer el fichero (el que hayamos elegido de forma arbitraria), pasar todos los símbolos a mayúsculas, eliminar el acento a las vocales acentuadas, eliminar los dígitos, caracteres de puntuación y espacios.

In [ ]:
# Imports globales
from unidecode import unidecode # Este módulo será de gran ayuda para el pre-tratamiento de los datos. Instaladlo si no lo tenéis
import numpy as np

In [ ]:
# Escribe tu codigo aqui!

## Preanálisis de la fuente de información

Ahora tenemos que definirnos unos métodos en nuestra clase para obtener la siguiente información:
- El alfabeto de nuestro texto (una lista con los caracteres).
- Dado un parámetro n, los n-gramas del texto.


In [ ]:
# Escribe tu codigo aqui!

_¿Cuáles son los bigramas presentes en el texto? ¿Hay menos de los que esperabas? ¿Por qué?_

Hay menos puesto que hay combinaciones de caracteres que no se suelen dar (o no se dan) en castellano. Una manera fácil de comprobar esto es ver que 

```python
len(fi.obtener_n_gramas(2)) <= (len(fi.alfabeto)**2)
```

## Fuente de Información de memoria nula

Nuestro siguiente paso es diseñar la fuente de información de memoria nula asociada al texto. Podéis representar la fuente de información como queráis, aunque yo os recomiendo que uséis un diccionario. Como probabilidades, debéis tomar las frecuencias de aparición de los caracteres en el texto.

In [ ]:
# Escribe tu codigo aqui!

## Fuente de Información de memoria nula extendida

Ahora vamos a diseñar la fuente de información de memoria nula extendida asociada al texto. En este caso, las probabilidades las podemos calcular como las frecuencias de aparición de los n-gramas en el texto. Para ello, tenemos que pasarle al constructor de la clase el parámetro n.

In [ ]:
# Escribe tu codigo aqui!

## Fuente de Markov

A continuación construiremos una fuente de Markov (de orden 2, para no perder mucho tiempo computacional) a partir del texto. Para ello hay que construir una matriz de probabilidades condicionales, donde las filas (y columnas) representan los bigramas del texto. Para el alfabeto usual (que tiene 27 caracteres), tendremos una matriz $27^2\times 27^2$, es decir 531441 entradas. Evidentemente, como algunos bigramas no aparecen, tendremos filas (y columnas) enteras que serán 0 (y las podremos eliminar para ahorrar espacio).

Para calcular las probabilidades de transición entre bigramas, obtendremos los trigramas que aparecen en el texto y después calcularemos las probabilidades de transición a partir de estos. Por ejemplo, el trigrama `XYZ` nos proporciona información de transición del bigrama `XY` al bigrama `YZ`. Por lo tanto, si obtenemos todos los trigramas que comienzan por `XY`, podremos obtener las frecuencias de transición desde `XY` a `YZ` para cualquier valor `Z`.

### Ejemplo

Supongamos que nuestro texto es `ABCBCBABBA`. Los trigramas que aparecen en el texto y su número de ocurrencia es:

| Trigrama | Conteo     |
|----------|------------|
| `ABC`    | 1          |
| `BCB`    | 2          |
| `CBC`    | 1          |
| `CBA`    | 1          |
| `BAB`    | 1          |
| `ABB`    | 1          |
| `BBA`    | 1          |

Podemos ver que hay dos trigramas que comienzan por AB (`ABC` y `ABB`) cada uno de ellos con una (1) aparición, por lo que las probabilidades de obtener `BC` a partir de `AB` es 0.5 (ya que de `AB` podemos pasar con la misma probabilidad a `ABC` o a `ABB`), y por ende la probabilidad de obtener `BB` a partir de `AB` sería 0.5. Con la anterior tabla,obtendríamos obtener la correspondiente matriz de probabilidades condicionales:


$$
\Pi = \begin{matrix}
& \begin{matrix} AB & BC & CB & BA & BB \end{matrix} \\
\begin{matrix} AB \\ BC \\ CB \\ BA \\ BB \end{matrix} & \begin{pmatrix} 0 & 0.5 & 0 & 0 & 0.5 \\ 0 & 0 & 1 & 0 & 0 \\ 0 & 0.5 & 0 & 0.5 & 0 \\ 1 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 & 0 \end{pmatrix}
\end{matrix} 
$$

Por lo tanto, para obtener esta matriz, tenemos que seguir el siguiente algoritmo:
1. Calcular los trigramas del texto y su número de apariciones
2. Para cada trigrama que comience por `XY`, calcularemos la probabilidad de `XY` a `YZ` de la siguiente forma:

$$
P(YZ|XY) = \frac{\text{Conteo}(XYZ)}{\text{Conteo}(XY\star)}
$$

donde $\star$ representa cualquier caracter.

In [ ]:
# Escribe tu codigo aqui!

## Obtención de Mensajes a partir de una Fuente de Markov

Vamos a implementar ahora un método que, dada nuestra fuente de Markov, emita un mensaje arbitrario. Esto lo hacemos de la siguiente manera:
- Elegimos un n-grama de forma aleatoria.
- Fijamos la longitud máxima del mensaje (`MAXLENGTH`).
- Elegimos un valor `m` de forma que `n <= m <= MAXLENGTH`.
- Partiendo del n-grama inicial, hay que elegir `m - n` n-gramas de forma secuencial, siguiendo la distribución de probabilidades de la matriz de transición.

### Ejemplo

Recordemos que nuestro ejemplo anterior tenía la siguiente matriz de transición:

$$
\Pi = \begin{matrix}
& \begin{matrix} AB & BC & CB & BA & BB \end{matrix} \\
\begin{matrix} AB \\ BC \\ CB \\ BA \\ BB \end{matrix} & \begin{pmatrix} 0 & 0.5 & 0 & 0 & 0.5 \\ 0 & 0 & 1 & 0 & 0 \\ 0 & 0.5 & 0 & 0.5 & 0 \\ 1 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 & 0 \end{pmatrix}
\end{matrix} 
$$

Elegimos (de forma arbitraria), una longitud de mensaje igual a 6. Elegimos de forma arbitraria un bigrama, por ejemplo el `BC`. A partir de aquí debemos elegir `6 - 2 = 4` bigramas adicionales de forma secuencial. 

Inicialmente, a partir de `BC` sólo podemos transitar a `CB`. A partir de `CB` podemos transitar con igual probabilidad a `BC` y `BA`; y elegimos `BA`. A partir de `BA` transitamos a `AB` y, por último, a partir de `AB` transitamos a `BC` (que hemos elegido arbitrariamente entre `BB` y `BC`). Por lo tanto, la secuencia de bigramas que hemos seguido es `BC`-`CB`-`BA`-`AB`-`BC` y el mensaje obtenido sería `BCBABC`.

In [ ]:
# Escribe tu codigo aqui!

## Obtención de la Entropía de un texto

Para terminar, vamos a definir un método que, dado un fichero de texto normalizado, obtenga su entropía.

En este caso, basta con calcular sobre el fichero la probabilidad de aparición de cada símbolo del alfabeto (lo cual ya hemos hecho). Por lo tanto, simplemente queda aplicar la definición de entropía:

$$
H(\mathcal{S}) = - \sum_{s\in\mathcal{S}}p(s)\log\left(p(s)\right)
$$

Realmente este caso sería para una fuente de memoria nula. Podemos extenderlo a fuentes de memoria nula de orden n con el resultado que hemos visto en clase de $H(\mathcal{S}^n)=nH(\mathcal{S})$

Además, usaremos nuestro generador de mensajes definido anteriormente para generar nuevos textos, y poder calcular su entropía.

In [ ]:
# Escribe tu codigo aqui!